In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
def plot_poi(df):
    c=df
    # c.drop(c.columns[[0,1]], axis=1, inplace=True) # 删除列
    #print(c)
    lat = c.loc[:,'lat']
    lon = c.loc[:,'lng']

    region_labels = get_region_label(lon, lat)
    print(region_labels)
    c.loc[:,'region'] = region_labels
    c = c[(c['region'] != 0) & (c['region'] != 1)]
    print(c)
    return c
    #c.to_csv('poi_total.csv', index=False)


In [3]:
def gps2xy(lon, lat):
    # GPS coordinate: 121.0047308 30.8856365最西 121.9724096 30.8997416最东 121.2702538 30.7056483最南 121.3004672 31.5001065最北
    # raster_loc = 1766 685最南 32 742最北 1340 1995最东 1365 192最西
    
    # calculate coefficient k and b
    """
    gpsX = np.array([[103.9925022, 104.088888], [688, 1729]])
    gpsY = np.array([[30.7097699, 30.590786], [856, 2144]])

    diffX = gpsX[:, 1] - gpsX[:, 0]
    kx = diffX[1] / diffX[0]
    bx = gpsX[1, 0] - gpsX[0, 0] * kx

    diffY = gpsY[:, 1] - gpsY[:, 0]
    ky = diffY[1] / diffY[0]
    by = gpsY[1, 0] - gpsY[0, 0] * ky
    print(kx, bx) 
    print(ky, by)
    """
    kx = 1896.250418888416 
    bx = -229280.27146698005
    ky = -2215.827560963539
    by = 69829.24459460145
 
 
    coordX = np.round(kx * lon + bx).astype(int)
    coordY = np.round(ky * lat + by).astype(int)
    # print(coordX, coordY)
    return (coordX, coordY) 
 
def get_region_label(lon, lat):
    map_table = np.loadtxt('region_labeled.csv')
    (coordX, coordY) = gps2xy(lon, lat)
    #print(coordX, coordY)
    labels = np.zeros(lon.shape[0])
    for k in range(len(coordX)):
        try:
            label = map_table[coordX[k], coordY[k]]
            labels[k] = label
        except IndexError:
            labels[k] = 0
    return labels.astype(int)

In [4]:
train = pd.read_csv(r'./data/house_rent.csv',encoding='gbk')
d = train.loc[:, ['price', 'lng', 'lat']]
d = d.dropna(axis=0)
d.to_csv('a.txt',index=None)
d2=pd.read_csv("a.txt")

In [10]:
dd=plot_poi(d2)


[  1   1   1 ... 117 116 116]
        price         lng        lat  region
142      2500  121.317299  31.250455       2
145      5000  121.321940  31.239219      98
261      4800  121.317500  31.237818      98
336      5200  121.317500  31.237818      98
438      2000  121.086483  31.300953      16
...       ...         ...        ...     ...
412070   6500  121.420756  31.202652     116
412071   5000  121.417172  31.204095     116
412072   4700  121.395441  31.208667     117
412073   7200  121.417172  31.204095     116
412074   7000  121.417213  31.205789     116

[311566 rows x 4 columns]


In [11]:
def poi_count(df):
    # feature matrix
    # every row is a region, except row 0 and 1
    # row0 is on the road
    # row1 is out of boundary
    points_vec = np.zeros((159,1))
    group = df.groupby('region')
    for key, values in group:
        # key is region number
        # freq is vector of 
        sum1=0
        k=0
        a=int(key)
        for index,row in values.iterrows():
            sum1+=row['price']
            k+=1

        points_vec[a, 0] = sum1/k
    print(points_vec[2:])
    return points_vec[2:]

In [12]:
aa=poi_count(dd)

[[ 7431.37879014]
 [    0.        ]
 [12000.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [ 3588.40752351]
 [ 3050.77578051]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [ 2915.80645161]
 [    0.        ]
 [    0.        ]
 [ 1582.22222222]
 [    0.        ]
 [    0.        ]
 [ 3033.59138973]
 [    0.        ]
 [ 2918.18181818]
 [    0.        ]
 [ 2944.10386473]
 [    0.        ]
 [ 2015.87301587]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [ 2000.        ]
 [    0.        ]
 [ 3297.82169118]
 [    0.        ]
 [ 4067.48275862]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [ 4450.        ]
 [ 4130.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [ 3027.98833819]
 [    0.        ]
 [ 4294.44444444]
 [40000.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [ 4805.4431946 ]
 [ 7764.28571429]
 [ 4712.00

In [18]:
dd.head()

,price,lng,lat,region
142,2500,121.317299,31.250455,2
145,5000,121.321940,31.239219,98
261,4800,121.317500,31.237818,98
336,5200,121.317500,31.237818,98
438,2000,121.086483,31.300953,16


In [19]:
len(ll)

87

In [23]:
df = pd.read_csv('out.txt',sep=' ',header=None)

df.loc[:,'price'] = aa
#print(df)
feature = df.iloc[ll]
feature.head()
feature.to_csv("feature.csv",index=0)

In [39]:
df = pd.read_csv(r'E:\Program Files\jupyter_code\pytorch\fea3.txt',sep=' ',header=None)

df.loc[:,'price'] = aa
#print(df)
feature = df.iloc[ll]
feature.head()
feature.to_csv("feature3_pre.csv",index=0)

In [25]:
df = pd.read_csv('out2.txt',sep=' ',header=None)

df.loc[:,'price'] = aa
#print(df)
feature = df.iloc[ll]
feature.head()
feature.to_csv("feature_pre.csv",index=0)

In [24]:
df2 = pd.read_csv('fea.txt',sep=' ',header=None)

df2.loc[:,'price'] = aa
#print(df)
feature2 = df2.iloc[ll]
feature2.head()
feature2.to_csv('feature2.csv',index=0)

In [28]:
df2 = pd.read_csv('fea2.txt',sep=' ',header=None)

df2.loc[:,'price'] = aa
#print(df)
feature2 = df2.iloc[ll]
feature2.head()
feature2.to_csv('feature2_pre.csv',index=0)

In [22]:
aa[ll,:]

array([[ 7431.37879014],
       [12000.        ],
       [ 3588.40752351],
       [ 3050.77578051],
       [ 2915.80645161],
       [ 1582.22222222],
       [ 3033.59138973],
       [ 2918.18181818],
       [ 2944.10386473],
       [ 2015.87301587],
       [ 2000.        ],
       [ 3297.82169118],
       [ 4067.48275862],
       [ 4450.        ],
       [ 4130.        ],
       [ 3027.98833819],
       [ 4294.44444444],
       [40000.        ],
       [ 4805.4431946 ],
       [ 7764.28571429],
       [ 4712.00203459],
       [ 3051.67115903],
       [ 8506.66967509],
       [ 4110.74688797],
       [ 4202.77777778],
       [ 3000.        ],
       [ 3696.49122807],
       [ 3198.10185185],
       [ 2750.        ],
       [ 4525.32288401],
       [ 4131.43202927],
       [ 6882.5957121 ],
       [ 1800.        ],
       [ 3686.14087369],
       [ 3993.14982578],
       [ 4299.07216495],
       [ 5381.94174757],
       [ 2307.        ],
       [ 3528.57142857],
       [10865.12400906],


In [26]:
out = np.loadtxt('out.txt')

In [28]:
out[ll,:]

array([[ 85., 889., 383., ...,  66.,  36.,  64.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  8., 150.,  87., ...,   3.,   5.,  16.],
       ...,
       [ 11., 168.,  47., ...,  13.,   8.,   9.],
       [  4.,  70.,  30., ...,   4.,   3.,   3.],
       [  0.,   1.,   5., ...,   0.,   0.,   1.]])

In [29]:
fea = np.loadtxt('fea.txt')
fea[ll,:]

array([[ 3.13261261e+02, -7.54627075e+01, -1.14751503e+02,
        -1.73242432e+02],
       [-3.14277411e-03, -2.48946436e-02,  2.64452398e-03,
        -1.75099801e-02],
       [ 4.61723862e+01, -1.19784460e+01, -1.78093739e+01,
        -2.64299412e+01],
       [ 3.67157516e+01, -9.77076340e+00, -1.44718828e+01,
        -2.14478722e+01],
       [ 1.97907257e+01, -4.22960997e+00, -6.80072260e+00,
        -1.03173990e+01],
       [ 4.67383802e-01, -1.29886568e-01, -1.57341585e-01,
        -2.72070765e-01],
       [ 1.06892014e+02, -2.72615337e+01, -4.11197319e+01,
        -6.08167191e+01],
       [ 1.75195808e+01, -4.48246765e+00, -6.70521116e+00,
        -9.95308304e+00],
       [ 2.24618168e+01, -4.62034559e+00, -7.62076902e+00,
        -1.15546675e+01],
       [ 5.78367090e+00, -1.44637954e+00, -2.17595172e+00,
        -3.24371529e+00],
       [ 7.15987625e+01, -1.79327602e+01, -2.70047913e+01,
        -4.03416328e+01],
       [ 5.69474449e+01, -1.37204561e+01, -2.12030010e+01,
      